In [12]:
import torch
import logging
import os
import io
import subprocess
import anvil.server
from PIL import Image
from anvil import BlobMedia
from utils import sr_utils

In [21]:
print(torch.cuda.get_device_properties(0).total_memory)
print(torch.cuda.memory_reserved(0))
print(torch.cuda.memory_allocated(0))

8589934592
69206016
66917888


In [14]:
model_name = "BSRGAN"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
logger = logging.getLogger("blind_sr_log")

In [15]:
model = sr_utils.load_model(model_name, device)

[3, 3, 64, 23, 32, 4]


In [17]:
anvil.server.connect(os.environ.get("anvil_uplink"))

In [19]:
@anvil.server.callable
def image_super_resolution(image):
    # save new image
    image = Image.open(io.BytesIO(image.get_bytes()))
    image.save("testsets/diffusion/diffusion.png", format="PNG")

    # apply super resolution
    sr_utils.super_resolution(model, logger, device)
    # subprocess.run(["python", "super_resolution.py"])

    # read image as bytes
    sr_image = Image.open("results/diffusion_results/diffusion_sr.png")
    output = io.BytesIO()
    sr_image.save(output, format="PNG")
    content = output.getvalue()

    # pass to anvil as a BlobMedia
    return BlobMedia("image/png", content)

In [22]:
anvil.server.wait_forever()

22-06-03 22:17:36.216 :           GPU ID : 0
22-06-03 22:17:36.219 :    1 --> diffusion.png


LogHandlers exist!


22-06-03 22:21:27.656 :           GPU ID : 0
22-06-03 22:21:27.661 :    1 --> diffusion.png


LogHandlers exist!


KeyboardInterrupt: 